## Minmax + Pruning

### + 状态评估，限制最大搜索深度

* 只做状态评估其实还是挺慢的，棋盘大小为5、深度为3的检索，就要很久了
* 也许是python实现的速度不太行

In [ ]:
# 环境配置
%cd ../../
import sys
sys.path.append('./python')

In [ ]:
from dlgobang import GameState, Player, Move, Point
from dlgobang.agent import DepthPrunedAgent
from dlgobang.utils import print_board, print_move, point_from_coords

In [ ]:
# 极简棋局状态评估，，简化胜负判定
def capture_diff_v1(game_state:GameState):
    black_score = 0
    white_score = 0
    
    # 只比较1-2-3-4连子数目， 威胁分分别为（1，3，10，20）
    smap = {1:1, 2:3, 3:10, 4:20}

    for line in game_state.board.lines.values():
        if game_state.next_player == Player.BLACK:
            black_score += smap[len(line)]
        else:
            white_score += smap[len(line)]
        
    diff = black_score - white_score
    
    if game_state.next_player == Player.BLACK:
        return diff
    return -1 * diff

### 人机博弈
搜索深度为3，复杂度类似与tic tac toe的每一步的递归判定。 单线程的ai-bot的落子速度很慢。

In [ ]:
BOARD_SIZE = 5
game = GameState.new_game(BOARD_SIZE)
bot = DepthPrunedAgent(3, capture_diff_v1) # 最大搜索深度3，仅通过棋子数的diff来判断棋面局势

while not game.is_over():
    print_board(game.board)
    if game.next_player == Player.BLACK:
        human_move = input('输入落子[A1-]:')
        point = point_from_coords(human_move.strip())
        move = Move.play(point)
    else:
        move = bot.select_move(game)
    print_move(game.next_player, move)
    game = game.apply_move(move)